Make video from photos in directory, based on photos, determined by time of recording

Print date and time on top of video

Indicate frames where the bee was rewarded

Loop through all directories in a folder and make videos!


Callin Switzer

11 Dec 2016

19 Dec 2016 -- Updated




In [58]:
import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

from datetime import datetime
import shutil

In [2]:
%qtconsole

In [3]:
import Tkinter as tk
import tkFileDialog

def file_choose():
    '''
    Opens tkfileDialog and returns full path name.
    It's the equivalent of file.choose in R.
    Make sure the following modeules are loaded
    import Tkinter as tk
    import tkFileDialog
    Also, won't work if %matplotlib qt4 has been used
    '''
    root = tk.Tk()
    root.update()
    file1 = tkFileDialog.askopenfilename(parent=root)
    root.destroy() # closes the window
    return file1

In [4]:
# adjust gamma
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [6]:
#file_choose()

'/Volumes/My Passport/BeeSonicationLearningWithAvery/BeeFrequencyLearning/BeeBlue_22Nov2016_Hive4_initial/2016_11_22__08_22_43_ampFreq.txt'

In [19]:

baseDir = "/Volumes/My Passport/BeeSonicationLearningWithAvery/BeeFrequencyLearning/"
os.chdir(baseDir)

In [27]:
beeTrialList =  [ f for f in os.listdir(".") if not f.startswith(r".") ]
beeTrialList[0:5]

['BeeBlue2_28Nov2016_Hive4_high',
 'BeeBlue3_28Nov2016_Hive4_high',
 'BeeBlue4_28Nov2016_Hive4_high',
 'BeeBlue5_28Nov2016_Hive4_high',
 'BeeBlue6_28Nov2016_Hive4_high']

In [182]:
# start of loop
trialNum = 5


os.chdir(baseDir)

CurrentTrialDir = os.path.join( os.getcwd() , beeTrialList[trialNum])
trialFolder = [ f for f in os.listdir(CurrentTrialDir) if not f.endswith(r".txt") and not f.startswith(r".") ]

trialMeta = [ f for f in os.listdir(CurrentTrialDir) if  f.endswith(r".txt") and not f.startswith(r".") ]
tm1 = pd.read_csv(os.path.join(CurrentTrialDir, trialMeta[0] ), header = None)

# replace leading spaces
tm1.ix[:,2] = [cc.replace(" ", "") for cc in tm1.ix[:,2]]

trialFolderDir = os.path.join(CurrentTrialDir, trialFolder[0])
photoList = [p for p in os.listdir(trialFolderDir) if p.endswith(r".pgm")]

# remove background photo from photolist
photoList.remove('background.pgm')

In [183]:
# sort list by date created
creDate = [os.stat(os.path.join(trialFolderDir, c)).st_mtime for c in  photoList]
creD_time = [datetime.fromtimestamp(aa).strftime('%Y-%m-%d %H:%M:%S') for aa in creDate]

sortedPhotos = np.array(photoList)[np.argsort(np.array(creDate))]

photoTimes = [c[:24] for c in sortedPhotos]



In [184]:
# rewards
rewArr = np.array(tm1.ix[:,2][tm1.ix[:,4] == " T"])

rewd1 = [photoTimes[kk] in rewArr for kk in range(len(photoTimes))]

In [185]:
# label rewards -- 
rr = np.cumsum(rewd1)

In [186]:
# make file with metadata from video
dd = pd.DataFrame({'photoName':sortedPhotos, 'photoTime':photoTimes,
                   'vidFrame':range(1, len(sortedPhotos) + 1), 
                  'dateModified':np.array(creD_time)[np.argsort(np.array(creDate))], 
                  'rewardNum':rr, 
                  'rewardTF': rewd1})
len(dd)




115

In [187]:
photoPaths = [os.path.join(trialFolderDir, c) for c in  sortedPhotos]
# change directory
newD = '/Users/callinswitzer/Desktop/CompressedBeeSonicationData/' + str(beeTrialList[trialNum]) + str(r"_compressed/")
if not os.path.isdir(newD):
    os.mkdir(newD)


In [188]:
# add text to images using opencv
# about 30 times faster than using matplotlib (below)
stt = time.time()
for kk in range(len(sortedPhotos)):

    # plt.clf()

    img = cv2.imread(photoPaths[kk], 0)


    s1 =  sortedPhotos[kk].split(".")[0]
    try:
        s2 = s1.split("__")
        s2[0] = s2[0].replace('_', '-')
        s2[1] = s2[1][:8].replace('_', ':') + '.' + s2[1][9:12]
        s3 = s2[0] + " " + s2[1]
    except:
        s3 = "No Timestamp"

    s4 = 'REWARD ' + str(rr[kk])
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, s3, (20, 60), font,  1.3, (0,0,0), thickness = 3) 

    if dd['rewardTF'][kk]:
        s4 = "REWARD " + str(rr[kk])
        cv2.putText(img, "REWARD " + str(rr[kk]), (20, 150), font,  1.3, (0,0,0), thickness = 3) 

    cv2.imwrite(newD + str(kk).zfill(4) + '.png', img)
    print str(kk) + ' of ' + str(len(sortedPhotos))
    
print time.time() - stt # get time to complete

# io.imshow(img)
  

0 of 115
1 of 115
2 of 115
3 of 115
4 of 115
5 of 115
6 of 115
7 of 115
8 of 115
9 of 115
10 of 115
11 of 115
12 of 115
13 of 115
14 of 115
15 of 115
16 of 115
17 of 115
18 of 115
19 of 115
20 of 115
21 of 115
22 of 115
23 of 115
24 of 115
25 of 115
26 of 115
27 of 115
28 of 115
29 of 115
30 of 115
31 of 115
32 of 115
33 of 115
34 of 115
35 of 115
36 of 115
37 of 115
38 of 115
39 of 115
40 of 115
41 of 115
42 of 115
43 of 115
44 of 115
45 of 115
46 of 115
47 of 115
48 of 115
49 of 115
50 of 115
51 of 115
52 of 115
53 of 115
54 of 115
55 of 115
56 of 115
57 of 115
58 of 115
59 of 115
60 of 115
61 of 115
62 of 115
63 of 115
64 of 115
65 of 115
66 of 115
67 of 115
68 of 115
69 of 115
70 of 115
71 of 115
72 of 115
73 of 115
74 of 115
75 of 115
76 of 115
77 of 115
78 of 115
79 of 115
80 of 115
81 of 115
82 of 115
83 of 115
84 of 115
85 of 115
86 of 115
87 of 115
88 of 115
89 of 115
90 of 115
91 of 115
92 of 115
93 of 115
94 of 115
95 of 115
96 of 115
97 of 115
98 of 115
99 of 115
100 of 115

In [189]:
oldDir = os.getcwd()

In [190]:
os.chdir(newD)
# use ffmpeg to convert directory to video
stta = time.time()
!ffmpeg -start_number 0 -r 24 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y trialVid.mp4
print time.time() - stta # ~ 1.4 seconds

ffmpeg version 3.1.4 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 8.0.0 (clang-800.0.38)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.1.4 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --disable-lzma --enable-vda
  libavutil      55. 28.100 / 55. 28.100
  libavcodec     57. 48.101 / 57. 48.101
  libavformat    57. 41.100 / 57. 41.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 47.100 /  6. 47.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  1.100 /  4.  1.100
  libswresample   2.  1.100 /  2.  1.100
  libpostproc    54.  0.100 / 54.  0.100
Input #0, image2, from '%04d.png':
  Duration: 00:00:04.60, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, gray(pc), 1280x1024, 25 fps, 25 tbr, 25 tbn, 25 tbc
[libx264 @ 0x7fbabb80ca00] usi

In [191]:
# delete all png files in directory
filelist = [ f for f in os.listdir(newD) if f.endswith(".png") ]

for f in filelist:
    os.remove(f)

In [192]:
# copy ampfreq to new directory
shutil.copy2(os.path.join(CurrentTrialDir, trialMeta[0] ), newD)



# END OF LOOP

In [ ]:
creD_time = [datetime.fromtimestamp(aa).strftime('%Y-%m-%d %H:%M:%S') for aa in creDate]
creD_time

In [ ]:
np.argsort(np.array(creDate))

In [ ]:
dd = pd.DataFrame({'file': np.array(photoList), 'time': np.array(creDate), 'time2':creD_time})
dd

In [ ]:
dd.ix[np.argsort(np.array(creDate)), :]

In [ ]:
sorted(photoList, reverse = False)

In [ ]:
x = np.array([3, 1, 2])
ind = np.argsort(x)

In [ ]:
x[ind]

In [ ]:
ind

In [181]:
## add text using matplotlib
# slower than opencv

stt = time.time()
for kk in range(len(sortedPhotos)):
    img = cv2.imread(photoPaths[kk], 0)


    s1 =  sortedPhotos[kk].split(".")[0]
    try:
        s2 = s1.split("__")
        s2[0] = s2[0].replace('_', '-')
        s2[1] = s2[1][:8].replace('_', ':') + '.' + s2[1][9:12]
        s3 = s2[0] + " " + s2[1]
    except:
        s3 = "No Timestamp"

    plt.clf()
    io.imshow(img)
    plt.axis([0, img.shape[1], img.shape[0], 0])
    plt.axis('off')

    plt.gca().set_position([0,0,1,1])
    plt.gca().set_aspect('equal')
    plt.text(850, 60, s3)
    #if s2[1]
    if dd['rewardTF'][kk]:
        plt.text(850, 90, "REWARD " + str(rr[kk]))


    # save figure
    plt.savefig(newD + str(kk).zfill(4) + '.png')
    print str(kk) + ' of ' + str(len(sortedPhotos))

print time.time() - stt



0 of 89
1 of 89
2 of 89
3 of 89
4 of 89
5 of 89
6 of 89
7 of 89
8 of 89
9 of 89
10 of 89
11 of 89
12 of 89
13 of 89
14 of 89
15 of 89
16 of 89
17 of 89
18 of 89
19 of 89
20 of 89
21 of 89
22 of 89
23 of 89
24 of 89
25 of 89
26 of 89
27 of 89
28 of 89
29 of 89
30 of 89
31 of 89
32 of 89
33 of 89
34 of 89
35 of 89
36 of 89
37 of 89
38 of 89
39 of 89
40 of 89
41 of 89
42 of 89
43 of 89
44 of 89
45 of 89
46 of 89
47 of 89
48 of 89
49 of 89
50 of 89
51 of 89
52 of 89
53 of 89
54 of 89
55 of 89
56 of 89
57 of 89
58 of 89
59 of 89
60 of 89
61 of 89
62 of 89
63 of 89
64 of 89
65 of 89
66 of 89
67 of 89
68 of 89
69 of 89
70 of 89
71 of 89
72 of 89
73 of 89
74 of 89
75 of 89
76 of 89
77 of 89
78 of 89
79 of 89
80 of 89
81 of 89
82 of 89
83 of 89
84 of 89
85 of 89
86 of 89
87 of 89
88 of 89
33.3139281273


In [ ]:
filelist = [ f for f in os.listdir(".") if f.endswith(".png") and not f.startwith(r".") ]